# Formação 2 - Machine Learning Development in Python
---
*João Brazuna*

## Apresentação dos Conjuntos de Dados
---
Nesta formação, usaremos maioritariamente dois conjuntos de dados reais e de acesso livre:

### [**Titanic**](https://www.kaggle.com/competitions/titanic/data)
<img src="images/Titanic.jpg" width="200">

Os dados do Titanic contêm informação real sobre os passageiros e a indicação se o passageiro sobreviveu ou não, que é aquilo que se pretende que seja previsto.

|  ID  | Variável    | Descrição                                 | Valores                                                              |
| ---: | :----       | :----                                     | :---                                                                 |
|    0 | PassengerId | ID do Passageiro                          | 1, 2, 3,..., 891                                                     |
|    1 | Survival    | Indicatriz da Sobrevivência do Passageiro | 1 se o passageiro tiver sobrevivido, 0 caso contrário                |
|    2 | Pclass      | Classe Económica do Passageiro            | 1 para 1ª classe, 2 para 2ª classe e 3 para passageiros de 3ª classe |
|    3 | Name        | Nome do Passageiro                        | Nome do passageiro, incluindo o título (Mr., Miss., Mrs.,...)        |
|    4 | Sex         | Género do Passageiro                      | "male" ou "female"                                                   |
|    5 | Age         | Idade do Passageiro em Anos               | 0.42, 0.67, 0.75, 0.83, 0.92, 1, 2,...                               |
|    6 | SibSp       | Número de Irmãos e Cônjuges no Titanic    | 0, 1, 2,...                                                          |
|    7 | Parch       | Número de Pais e Filhos no Titanic        | 0, 1, 2,...                                                          |
|    8 | Ticket      | Número do Bilhete                         | "A/5 21171", "PC 17599", "STON/O2. 3101282",...                      |
|    9 | Fare        | Preço do Bilhete                          | 0, 4.0125, 5, 6.2375,...                                             |
|   10 | Cabin       | Camarote onde ficou o Passageiro          | "C85", "C123", "E46",...                                             |
|   11 | Embarked    | Porto onde o Passageiro Embarcou          | "C" para Cherbourg, "Q" para Queenstown ou "S" para Southampton      |

## Importar Pacotes Necessários
---

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression

import statsmodels.api as sm

## Carregar os Dados
---

In [7]:
titanic_df = pd.read_csv('data/Titanic.csv')
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
class ConfusionMatrix():
    def __init__(self, y_real, y_pred):
        if (len(np.intersect1d(y_real, [1, 0])) != 2) or len(y_real.unique()) != 2:
            raise ValueError(f"y_real parameter can only take levels 1 or 0.")
        cm = confusion_matrix(y_real, y_pred, labels=[1, 0])
        cm_df = pd.DataFrame(cm, index=['Real Positive', 'Real Negative'], columns=['Predicted Positive', 'Predicted Negative'])
        self._confusion_matrix = cm_df
    
    @property
    def confusion_matrix(self):
        return self._confusion_matrix
    
    @property
    def tp(self):
        return self.confusion_matrix.loc['Real Positive', 'Predicted Positive']
    
    @property
    def tn(self):
        return self.confusion_matrix.loc['Real Negative', 'Predicted Negative']
    
    @property
    def fp(self):
        return self.confusion_matrix.loc['Real Negative', 'Predicted Positive']
    
    @property
    def fn(self):
        return self.confusion_matrix.loc['Real Positive', 'Predicted Negative']
    
    @property
    def positive(self):
        return self.confusion_matrix.loc['Real Positive', :].sum()
    
    @property
    def negative(self):
        return self.confusion_matrix.loc['Real Negative', :].sum()
    
    @property
    def predicted_positive(self):
        return self.confusion_matrix.loc[:, 'Predicted Positive'].sum()
    
    @property
    def predicted_negative(self):
        return self.confusion_matrix.loc[:, 'Predicted Negative'].sum()
    
    ## Performance Measures ##
    @property
    def accuracy(self):
        return (self.tp + self.tn) / (self.positive + self.negative)
    
    # Sensibility
    @property
    def sensibility(self):
        return self.tp / self.positive
    
    @property
    def recall(self):
        return self.sensibility
    
    @property
    def tpr(self):
        return self.sensibility
    
    # Specificity
    @property
    def specificity(self):
        return self.tn / self.negative
    
    @property
    def selectivity(self):
        return self.specificity
    
    @property
    def tpr(self):
        return self.specificity
    
    # False Positive Rate
    @property
    def fpr(self):
        return 1 - self.specificity
    
    # False Negative Rate
    @property
    def fnr(self):
        return 1 - self.sensitivity
    
    # PPV / Precision
    @property
    def precision(self):
        return self.tp / self.predicted_positive
    
    @property
    def ppv(self):
        return self.precision
    
    # NPV
    @property
    def npv(self):
        return self.tn / self.predicted_negative
    
    # False Discovery Rate
    @property
    def fdr(self):
        return 1 - self.ppv
    
    # False Omission Rate
    @property
    def fomr(self):
        return 1 - self.npv
    
    # Performance Measures
    @property
    def performance_measures(self):
        pm_dct = {'Accuracy': self.accuracy,
                  'Precision / PPV': self.precision,
                  'Recall / Sensibility': self.sensibility,
                  'NPV': self.npv,
                  'Specificity': self.specificity}
        return pd.Series(pm_dct).to_frame('Performance Measures').T
    
    
def target_by_feature(x, target, data, fun='mean', bins=None,
                      xlabel=None, ylabel=None, title=None, targetlabels=None, 
                      xrotation=None, show=True):
    df = data.copy()
    df['Number of Observations'] = 1
    
    if isinstance(target, str):
        targets = [target]
    elif isinstance(target, list) and all([isinstance(y, str) for y in target]):
        targets = target
    else:
        raise ValueError(f"target argument can only be a string or a list of strings")
    
    if targetlabels is None:
        targetlabels = targets
    elif (isinstance(targetlabels, str) and len(targets) == 1):
        targetlabels = [targetlabels]
    elif ((isinstance(targetlabels, str) and len(targets) != 1)) or (isinstance(targetlabels, list) and (len(targetlabels) != len(targets))):
        raise ValueErrors(f"targetlabels argument must be a string if target is a string or a singleton list, or a list with the same number of elements as target.")

    if bins is not None:
        if isinstance(bins, list):
            if len(bins) != 3:
                raise ValueError(f"When a list, bins must be have 3 elements")
            bins = [df[x].min() - bins[2], *np.arange(bins[0], bins[1], bins[2]).tolist(), df[x].max()]     
            n_bins = len(bins)
        elif isinstance(bins, int):
            n_bins = bins
            
        unique_vals = df[x].unique()
        if len(unique_vals) > n_bins:
            df[x] = pd.cut(df[x], bins=bins)#.apply(lambda x: x.right)
    
    agg_funs = {'Number of Observations': 'sum'}
    for target in targets:
        agg_funs[target] = fun
    
    grp_df = df.groupby(x).agg(agg_funs).sort_index()
    grp_df['Number of Observations (%)'] = grp_df['Number of Observations'] / grp_df['Number of Observations'].sum() * 100
    grp_df.reset_index(inplace=True)
    
    ax = sns.barplot(x=x, y='Number of Observations (%)', data=grp_df, color='grey', alpha=0.4)
    ax.yaxis.set_major_formatter(PercentFormatter(decimals=0))
    ylims = ax.get_ylim()
    y_adj = (ylims[1] - ylims[0]) / 100
    
    for ii, p in enumerate(ax.patches):
        txt = str(int(grp_df.iloc[ii]['Number of Observations']))
        txt_x = p.get_x() + p.get_width() / 2
        txt_y = p.get_height() + y_adj
        ax.text(txt_x, txt_y, txt, ha='center')
    
    ax2 = ax.twinx()
    
    for target in targets:
        sns.lineplot(x=ax.get_xticks(), y=target, data=grp_df, ax=ax2)
    
    if xlabel is None:
        xlabel = x.title()
    ax.set_xlabel(xlabel)
    
    preffix = 'Average ' if fun in ['mean', np.mean] else 'Sum ' if fun in ['sum', np.sum] else ''
    if ylabel is None:
        ylabel = preffix + 'Target Value' 
    ax2.set_ylabel(ylabel)
    
    if title is None:
        title = ylabel + ' by ' + xlabel
    ax.set_title(title)
    
    plt.legend(labels=targetlabels, bbox_to_anchor=(1.7,1))
    
    if xrotation is not None:
        ax.tick_params(axis='x', rotation=90)
    if show:
        plt.show()

## Regressão Logística - Prever a Probabilidade de Sobrevivência

In [9]:
numeric_vars = titanic_df.dtypes[titanic_df.dtypes != 'object'].index.to_list()

In [10]:
X = titanic_df[numeric_vars].drop(['PassengerId', 'Pclass'], axis=1).dropna(axis=1) #passenger  id and pclass out
y = pd.get_dummies(titanic_df['Pclass'])[1] #1, 2 ou 3
y

0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: 1, Length: 891, dtype: uint8

In [11]:
lm = sm.Logit(y, sm.add_constant(X))
res = lm.fit()
res.summary()

Optimization terminated successfully.
         Current function value: inf
         Iterations 9


d:\Users\user\anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
d:\Users\user\anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
d:\Users\user\anaconda3\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
d:\Users\user\anaconda3\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      1   No. Observations:                  891
Model:                          Logit   Df Residuals:                      886
Method:                           MLE   Df Model:                            4
Date:                Thu, 20 Apr 2023   Pseudo R-squ.:                     inf
Time:                        20:42:07   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.5792      0.318    -14.401      0.000      -5.202      -3.956
Survived       0.5315      0.306      1.738      0.082      -0.068       1.131
SibSp         -2.8835      0.487     -5.921      0.000      -3.838      -1.929
Parch         -1.6190      0.386     -4.198      0.000      -2.375      -0.863
Fare           0.1578      0.013     11.992      0.000       0.132       0.184
==============================================================================

Possibly complete quasi-separation: A fraction 0.11 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [12]:
lm2 = LogisticRegression(random_state=123)
lm2.fit(X, y)
lm2.score(X, y)

0.936026936026936

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 4) (179, 4) (712,) (179,)


In [14]:
lm2.fit(X_train, y_train)
lm2.score(X_train, y_train)

0.9297752808988764

In [15]:
lm2.score(X_test, y_test)

0.9217877094972067

In [16]:
confusion_matrix(y_test, lm2.predict(X_test), labels=[1, 0])

array([[ 38,  12],
       [  2, 127]], dtype=int64)

In [17]:
(35 + 90) / (35 + 90 + 24 + 30)

0.6983240223463687

In [18]:
y_pred = lm2.predict(X_test)

In [19]:
cm = ConfusionMatrix(y_test, lm2.predict(X_test))
cm.confusion_matrix.T

,Real Positive,Real Negative
Predicted Positive,38,2
Predicted Negative,12,127


In [20]:
cm.performance_measures

,Accuracy,Precision / PPV,Recall / Sensibility,NPV,Specificity
Performance Measures,0.921788,0.95,0.76,0.913669,0.984496
